In [2]:
import random
import pennylane as qml
from pennylane import numpy as np
from myutils import Datasets
from myutils import Preprocessing
from myutils import Helpers
import os
import sys
sys.path.insert(0,'..')
from maskit.datasets import load_data
from matplotlib import pyplot as plt

#Magic Command, so changes in myutils module are reloaded
%load_ext autoreload
%autoreload 1
%aimport myutils

In [3]:
# Setting seeds for reproducible results
np.random.seed(1337)
random.seed(1337)

# Loading the data

Data of interest is MNIST data. As we want to go for reproducible results, we
will first go with the option `shuffle=False`. For the rest of the parameters,
we now go with the default options. This gives us data for two classes, the
written numbers 6 and 9. We also only get a limited number of sampes, that is
100 samples for training and 50 for testing. For further details see the
appropriate docstring.

In [4]:
data=  Datasets.get_preprocessed_datasets("Autoencoder_6Epochs")["Autoencoder_6Epochs"]["6,9"]

In [5]:
data=  Datasets.get_preprocessed_datasets("PCA")["PCA"]["6,9"]

#scale data 0, 2*np
data["x_train"] = Preprocessing.minmax_scaler(data["x_train"] , min = 0,max = 2 * np.pi)
data["x_test"] = Preprocessing.minmax_scaler(data["x_test"], min = 0,max = 2 * np.pi)

for type in ["y_train_binary","y_test_binary"]:
    quantum_convert = []
    for i in range(len(data[type])):
        if data[type][i] == 0:
            quantum_convert.append(np.array([0,1]))
        else:
            quantum_convert.append(np.array([1,0]))

    data[type+"_quantum"] = np.array(quantum_convert)

n = 200
split = 0.75
data["x_train"] = data["x_train"][:int(n*split)]
data["y_train_binary_quantum"] = data["y_train_binary_quantum"][:int(n*split)]
data["x_test"] = data["x_test"][:int(n*(1-split))]
data["y_test_binary_quantum"] = data["y_test_binary_quantum"][:int(n*(1-split))]

print("train {}, test{}".format(int(n*split),int(n*(1-split))))

train 150, test50


In [6]:
for type in ["y_train_binary","y_test_binary"]:
    quantum_convert = []
    for i in range(len(data[type])):
        if data[type][i] == 0:
            quantum_convert.append(np.array([0,1]))
        else:
            quantum_convert.append(np.array([1,0]))

    data[type+"_quantum"] = np.array(quantum_convert)

In [7]:
n = 200
split = 0.75
data["x_train"] = data["x_train"][:int(n*split)]
data["y_train_binary_quantum"] = data["y_train_binary_quantum"][:int(n*split)]
data["x_test"] = data["x_test"][:int(n*(1-split))]
data["y_test_binary_quantum"] = data["y_test_binary_quantum"][:int(n*(1-split))]

print("train {}, test{}".format(int(n*split),int(n*(1-split))))

train 150, test50


# Setting up a Variational Quantum Circuit for training

There is an example on the [PennyLane website](https://pennylane.ai/qml/demos/tutorial_variational_classifier.html#iris-classification) for iris data showing a setup for a variational classifier. That is variational quantum circuits that can be trained from labelled (classical) data.

In [34]:
wires = 4
layers = 4
epochs = 5
parameters = np.random.uniform(low=-np.pi, high=np.pi, size=(layers, wires, 2))

In [35]:
def variational_circuit(params):
    for layer in range(layers):
        for wire in range(wires):
            qml.RX(params[layer][wire][0], wires=wire)
            qml.RY(params[layer][wire][1], wires=wire)
        for wire in range(0, wires - 1, 2):
            qml.CZ(wires=[wire, wire + 1])
        for wire in range(1, wires - 1, 2):
            qml.CZ(wires=[wire, wire + 1])
    return qml.expval(qml.PauliZ(0))

In [36]:
def variational_training_circuit(params, data):
    qml.templates.embeddings.AngleEmbedding(
        features=data, wires=range(wires), rotation="X"
    )
    return variational_circuit(params)

In [37]:
dev = qml.device('default.qubit', wires=wires, shots=1000)
circuit = qml.QNode(func=variational_circuit, device=dev)
training_circuit = qml.QNode(func=variational_training_circuit, device=dev)

In [38]:
circuit(parameters)

tensor(-0.052, requires_grad=True)

In [39]:
drawer = qml.draw(circuit)
print(drawer(parameters))

 0: ──RX(-1.5)───RY(-2.14)───╭C───RX(1.46)────RY(-2.42)──────────────╭C───RX(1.85)───RY(-0.872)─────────────╭C───RX(-0.00221)──RY(-2.02)──────────────╭C──────┤ ⟨Z⟩ 
 1: ──RX(-1.39)──RY(-0.256)──╰Z──╭C───────────RX(-0.715)──RY(0.807)──╰Z──╭C──────────RX(-0.527)──RY(0.529)──╰Z──╭C─────────────RX(-0.546)──RY(-1.89)──╰Z──╭C──┤     
 2: ──RX(-1.12)──RY(0.116)───╭C──╰Z───────────RX(-2.36)───RY(3.04)───╭C──╰Z──────────RX(1.63)────RY(-1.96)──╭C──╰Z─────────────RX(0.199)───RY(2.09)───╭C──╰Z──┤     
 3: ──RX(-1.5)───RY(2.99)────╰Z───RX(-0.357)──RY(1.82)───────────────╰Z───RX(-1.33)──RY(1.07)───────────────╰Z───RX(-1.98)─────RY(2.87)───────────────╰Z──────┤     



In [40]:
# some helpers
def correctly_classified(params, data, target):
    prediction = training_circuit(params, data)
    if prediction < 0 and target[0] > 0:
        return True
    elif prediction > 0 and target[1] > 0:
        return True
    return False

def overall_cost_and_correct(cost_fn, params, data, targets):
    cost = correct_count = 0
    for datum, target in zip(data, targets):
        cost += cost_fn(params, datum, target)
        correct_count += int(correctly_classified(params, datum, target))
    return cost, correct_count

In [41]:
# Playing with different cost functions
def crossentropy_cost(params, data, target):
    prediction = training_circuit(params, data)
    scaled_prediction = prediction + 1 / 2
    predictions = np.array([1 - scaled_prediction, scaled_prediction])
    return cross_entropy(predictions, target)

def distributed_cost(params, data, target):
    """Cost function distributes probabilities to both classes."""
    prediction = training_circuit(params, data)
    scaled_prediction = prediction + 1 / 2
    predictions = np.array([1 - scaled_prediction, scaled_prediction])
    return np.sum(np.abs(target - predictions))

def cost(params, data, target):
    """Cost function penalizes choosing wrong class."""
    prediction = training_circuit(params, data)
    predictions = np.array([0, prediction]) if prediction > 0 else np.array([prediction * -1, 0])
    return np.sum(np.abs(target - predictions))

In [42]:
optimizer = qml.AdamOptimizer()
cost_fn = distributed_cost

In [45]:
start_cost, correct_count = overall_cost_and_correct(cost_fn, parameters, data["x_test"], data["y_test_binary_quantum"])
print("start cost: {}, with {}/{} correct samples".format(start_cost,correct_count,len(data["y_test_binary_quantum"])))

start cost: 32.647999999999996, with 38/50 correct samples


In [178]:
params = parameters.copy()
for _ in range(epochs):
    print(_)
    for datum, target in zip(data_new["x_train_pca"], data_new["y_test_binary_quantum"]):
        params = optimizer.step(lambda weights: cost_fn(weights, datum, target), params)

    cost, correct_count = overall_cost_and_correct(cost_fn, params, data_new["x_test_pca"], data_new["y_test_binary_quantum"])
    print("cost: {}, with {}/{} correct samples".format(cost,correct_count,len(data_new["y_test_binary_quantum"])))

0
cost: 40.616, with 2/30 correct samples
1
cost: 36.952000000000005, with 2/30 correct samples
2
cost: 33.104, with 9/30 correct samples
3
cost: 30.188000000000002, with 17/30 correct samples
4
cost: 29.412, with 17/30 correct samples


In [179]:
final_cost, correct_count = overall_cost_and_correct(cost_fn, params, data.test_data, data.test_target)
print(f"final cost: {final_cost}, with {co rrect_count}/{len(data.test_target)} correct samples")

final cost: 55.660000000000004, with 20/50 correct samples
